In [151]:
%matplotlib inline
from pylab import *
import pandas as pd
from sklearn.decomposition import PCA
import yaml

In [152]:
def sort_lidar_file_and_shape(lidar_file_name, yaml_file_name, width=2088, height=64):
    df = pd.read_csv(lidar_file_name,
                     names = ['x', 'y', 'z', 'intensity', 'ring', 'rotation', 'revolution'])
    
    yaml_file = yaml.load(open(yaml_file_name))
    
    calibration = yaml_file['lasers']
    
    sorted_lasers = sorted(calibration, key=lambda x: x['vert_correction'], reverse=True)

    for i in range(0, 64):
        df.loc[df['ring'] == i, 'rotation'] = (df.loc[df['ring'] == i, 'rotation'] 
                                               + sorted_lasers[i]['rot_correction'] * 18000 / np.pi)
    
    df.loc[df['rotation'] > 36000, 'rotation'] -= 36000
    df.loc[df['rotation'] < 0, 'rotation'] += 36000
    
    img = np.zeros((height, width, 3))
    for i in range(height):
        img[i] = df.loc[df['ring'] == i].sort_values(['rotation']).as_matrix()[:, :3]
    
    return img

In [153]:
readings_file_name = "1509818706.038833141-cloudpoint.csv"
calibration_file_name = "64HDL_S2.yaml"

lidar_table = sort_lidar_file_and_shape(readings_file_name, calibration_file_name)

width = 2088
height = 64 
lidar = lidar_table.reshape((width * height, 3))

In [154]:
#dists = np.linalg.norm(lidar, axis=-1)
#inds = np.where(dists < 10)

inds = np.arange(len(lidar))
 
lidar_center = lidar[inds]

green = np.array([0, 1, 0])
red = np.array([1, 0, 0])
blue = np.array([0, 0, 1])

In [155]:
lidar_file_name = "lidar_test.csv"

with open(lidar_file_name, 'w') as f:
    for rd in lidar_center:
        f.write("{}, {}, {}\n".format(rd[0], rd[1], rd[2]))

In [156]:
order = np.zeros((height, width, 3))
half_width = width / 2
for i in range(width):
    for j in range(height):
        x = i
#         if x >= half_width:
#             x = width - i
#         rgval = x / half_width
        rgval = x / width
        bval = j / height
        order[j, i] = rgval * red + (1 - rgval) * green + bval * blue

order = order.reshape((width * height, 3))

order_center = order[inds]

with open("width_order_colors_test.csv", 'w') as f:
    for rd in order_center:
        f.write("{}, {}, {}\n".format(rd[0], rd[1], rd[2]))

In [157]:
depths = np.linalg.norm(lidar_table, axis=-1)

In [158]:
normals = np.zeros(lidar_table.shape)
neighbor_y_radius = 3

neighbor_x_radius = 3

for i in range(neighbor_y_radius, height-neighbor_y_radius - 1):
    for j in range(0, width):        
        neighbor_y_start = i - neighbor_y_radius
        neighbor_y_end = i + neighbor_y_radius + 1
        neighbor_x = np.arange(j - neighbor_x_radius, j + neighbor_x_radius + 1)
        
        neighbors = lidar_table[neighbor_y_start:neighbor_y_end]
        neighbors = np.take(neighbors, neighbor_x, axis = 1, mode='wrap')
        neighbors = neighbors.reshape(((neighbor_y_radius * 2 + 1) * (neighbor_x_radius * 2 + 1), 3))
        
        neighbor_depths = depths[neighbor_y_start:neighbor_y_end]
        neighbor_depths = np.take(neighbor_depths, neighbor_x, axis=1, mode='wrap')
        neighbor_depths = neighbor_depths.ravel()
        
        valid_neighbors = neighbors[np.abs(neighbor_depths - depths[i, j]) < 0.2]
        if valid_neighbors.shape[0] >= 3:
            covariance = np.cov(valid_neighbors, rowvar=False)
            eigvals, eigvecs = np.linalg.eigh(covariance)
            normals[i, j] = eigvecs[:, 0]
            normals[i, j] /= np.linalg.norm(normals[i, j])
            if np.dot(normals[i, j], np.array([0, 1, 0]) - 
                  lidar_table[i, j]) <= 0:
                normals[i, j] = -normals[i, j]

normal_img = normals 

In [159]:
normal_ply_file_name = "normals.ply"
normal_colors = normal_img.reshape((normal_img.shape[0] * normal_img.shape[1], 3))
normal_points = lidar_table.reshape((lidar_table.shape[0] * lidar_table.shape[1], 3))
with open(normal_ply_file_name, 'w') as f:
    f.write("ply\n")
    f.write("format ascii 1.0\n")
    f.write("element vertex {}\n".format(len(normal_points)))
    f.write("property float32 x\n")
    f.write("property float32 y\n")
    f.write("property float32 z\n")
    f.write("property float32 nx\n")
    f.write("property float32 ny\n")
    f.write("property float32 nz\n")
    f.write("end_header\n")
    for i in range(len(normal_points)):
        normal = normal_colors[i]
        f.write("{} {} {} {} {} {}\n".format(normal_points[i][0], normal_points[i][1], normal_points[i][2],
                                             normal[0], normal[1], normal[2]))

In [176]:
class Cluster:
    def __init__(self, normal, num):
        self.count = 1
        self.normal = normal
        self.id = num
        self.points = set()
        
    def add_normal(self, normal):
        self.normal = (self.normal * self.count + normal) / (self.count + 1)
        self.count += 1
        
    def remove_normal(self, normal):
        self.normal = (self.normal * self.count - normal) / (self.count - 1)

In [177]:
cluster_assignments = [ [Cluster(normal_img[i, j], i * normal_img.shape[1] + j) 
                         for j in range(normal_img.shape[1])]
                       for i in range(normal_img.shape[0]) ]
id_to_cluster = []
for row in cluster_assignments:
    for cluster in row:
        id_to_cluster.append(cluster)

cluster_id_to_coords = [[(i // normal_img.shape[1], i % normal_img.shape[1])]
                        for i in range(normal_img.shape[0] * normal_img.shape[1])]
            
while len(to_check) > 0:
    merges = dict()
    remove_set = set()
    
    for coord in to_check:
        x = coord[0]
        y = coord[1]
        
        cluster = cluster_assignments[x][y]
        normal = cluster.normal
        new_cluster = None
        
        for dx in range(-1, 2):
            for dy in range(-1, 2):
                new_x = x + dx
                if new_x >= normal_img.shape[0]:
                    new_x -= normal_img.shape[0]
                elif new_x < 0:
                    new_x += normal_img.shape[0]
                new_y = y + dy
                
                temp_cluster = cluster_assignments[new_x][new_y]
                if temp_cluster.id != cluster.id:
                    merge = (min(temp_cluster.id, cluster.id), max(temp_cluster.id, cluster.id)) 
                    
                    angle = np.arccos(np.dot(normal, temp_cluster.normal) / 
                                      np.linalg.norm(normal) / np.linalg.norm(temp_cluster.normal))
                    diff = np.abs(depths[new_x, new_y] - depths[x, y])
                    
                    if merge not in merges:
                        if diff < 0.2 and angle < np.pi / 6:
                            merges[merge] = angle
                    elif diff < 0.2:
                        merges[merge] = min(angle, merges[merge])
                        
    print(len(merges))
    if len(merges) == 0:
        break
        
    potential_merges = sorted(merges.keys(), key = lambda x: x[0])
    current_cluster = potential_merges[0][0]
    min_merge_candidate = potential_merges[0][1]
    min_merge_angle = np.pi
    for i in range(1, len(potential_merges) + 1):
        if i == len(potential_merges) or potential_merges[i][0] != current_cluster:
            coords = cluster_id_to_coords[current_cluster]
            cluster_id_to_coords[current_cluster] = []
            new_cluster = id_to_cluster[min_merge_candidate]
            for coord in coords:
                new_cluster.add_normal(normal_img[coord[0]][coord[1]])
                cluster_assignments[coord[0]][coord[1]] = new_cluster
            cluster_id_to_coords[min_merge_candidate] += coords
            min_merge_angle = np.pi
            if i < len(potential_merges) - 1:
                min_merge_candidate = potential_merges[i + 1][1]
                current_cluster = potential_merges[i + 1][0]
        else:
            if merges[potential_merges[i]] < min_merge_angle:
                min_merge_candidate = potential_merges[i][1]
                min_merge_angle = merges[potential_merges[i]]        

/Users/garychen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/Users/garychen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in arccos


312545
24833
9308
4509
2276
1188
618
319
166
88
46
24
12
6
3
1
0


In [178]:
valid_cluster_assignments = cluster_assignments[neighbor_y_radius:height-neighbor_y_radius - 1]
valid_lidars = lidar_table[neighbor_y_radius:height-neighbor_y_radius - 1]

# Add points to clusters
valid_cluster_ids = set()
for i in range(len(valid_cluster_assignments)):
    for j in range(len(valid_cluster_assignments[i])):
        point = valid_lidars[i, j]
        cluster = valid_cluster_assignments[i][j]
        cluster.points.add(point)
        valid_cluster_ids.add(cluster.id)

# Iterate through clusters and perform convex hull algorithm to get minimum bounding rectangle


TypeError: unhashable type: 'numpy.ndarray'

In [171]:
import random

blue = np.array([0, 0, 1])
red = np.array([1, 0, 0])
ids = []

for row in valid_cluster_assignments:
    for cluster in row:
        ids.append(cluster.id)
        
max_id = max(ids)
colors = dict()

for i in ids:
    colors[i] = (random.random(), random.random(), random.random())

with open("cluster_points.csv", 'w') as f1, open("cluster_colors.csv", 'w') as f2:
    lidars = valid_lidars.reshape((valid_lidars.shape[0] * valid_lidars.shape[1], 3))
    for i in range(len(ids)):
        f1.write("{}, {}, {}\n".format(lidars[i][0], lidars[i][1], lidars[i][2]))
        color = colors[ids[i]]
        f2.write("{}, {}, {}\n".format(color[0], color[1], color[2]))